In [1]:
import numpy as np
import pandas as pd
import os
path = os.getcwd()
pd.set_option("max_columns",0)

In [2]:
df_inst = pd.read_csv('./csv/installments_payments.csv')
df_pos = pd.read_csv('./csv/POS_CASH_balance.csv')
df_prev = pd.read_csv('./csv/previous_application.csv')
df_bu = pd.read_csv('./csv/bureau.csv')
df_app = pd.read_csv('./csv/application_train.csv')

### **Unite the Home Credit files:**

In [3]:
# num of unique SK_ID_PREV in each file
print(df_inst['SK_ID_PREV'].nunique(),df_pos['SK_ID_PREV'].nunique(),df_prev['SK_ID_PREV'].nunique())
# convet to set
set_inst = set(df_inst['SK_ID_PREV'])
set_pos = set(df_pos['SK_ID_PREV'])
set_prev = set(df_prev['SK_ID_PREV'])
# intersection of sets
set_inter_idx = set_inst.intersection(set_pos, set_prev)
# num of SK_ID_PREV in the intersection set
print(len(set_inter_idx))
# create new files
df_inst_new = df_inst[df_inst['SK_ID_PREV'].isin(set_inter_idx)]
df_pos_new = df_pos[df_pos['SK_ID_PREV'].isin(set_inter_idx)]
df_prev_new = df_prev[df_prev['SK_ID_PREV'].isin(set_inter_idx)]

997752 936325 1670214
896302


### **application_train.csv**

In [4]:
#the columns that we decided that are relevant to our model
df = df_app
df_select1 = df.iloc[:, 0:19]
df_select1.drop(['NAME_TYPE_SUITE', 'REGION_POPULATION_RELATIVE',  'NAME_CONTRACT_TYPE', 'AMT_GOODS_PRICE'], axis=1, inplace=True)
df_select2 = df.iloc[:, 31]
df_select3 = df.iloc[:, -27:-26]
df_selected = pd.concat([df_select1, df_select2, df_select3], axis=1)

In [5]:
df_selected.columns

Index(['SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'REGION_RATING_CLIENT_W_CITY',
       'DAYS_LAST_PHONE_CHANGE'],
      dtype='object')

In [6]:
# Categorial Reduction - unified analogous categories in 4 features:
# NAME_INCOME_TYPE== NAME_EDUCATION_TYPE==  NAME_FAMILY_STATUS==  NAME_HOUSING_TYPE
name_types = {'Working':'Working',
              'Commercial associate':'Commercial associate',
              'Pensioner':'Pensioner',
              'State servant':'Working',
              'Unemployed':'Unemployed',
              'Student':'Unemployed',
              'Businessman':'Commercial associate',
              'Maternity leave':'Working'}
df_selected.loc[:, 'NAME_INCOME_TYPE'] = df_selected.NAME_INCOME_TYPE.replace(name_types)

education_types = {'Secondary / secondary special':'Secondary',
              'Higher education':'Higher education',
              'Incomplete higher':'Incomplete higher',
              'Lower secondary':'Secondary',
              'Academic degree':'Higher education'}
df_selected.loc[:, 'NAME_EDUCATION_TYPE'] = df_selected.NAME_EDUCATION_TYPE.replace(education_types)

family_status = {'Married':'Married',
              'Single / not married':'not married',
              'Civil marriage':'Married',
              'Separated':'not married',
              'Widow':'Widow',
              'Unknown':'Married'}
df_selected.loc[:, 'NAME_FAMILY_STATUS'] = df_selected.NAME_FAMILY_STATUS.replace(family_status)

housing_type = {'House / apartment':'House / apartment',
              'With parents':'With parents',
              'Municipal apartment':'Municipal apartment',
              'Rented apartment':'Rented apartment',
              'Office apartment':'Rented apartment',
              'Co-op apartment':'House / apartment'}
df_selected.loc[:, 'NAME_HOUSING_TYPE'] = df_selected.NAME_HOUSING_TYPE.replace(housing_type)

In [7]:
# Binary Features = converted to 0,1
gender = {'M':0, 'F':1, 'XNA': 1}
df_selected.loc[:, 'CODE_GENDER'] = df_selected.CODE_GENDER.replace(gender)

car = {'Y':0, 'N':1}
df_selected.loc[:, 'FLAG_OWN_CAR'] = df_selected.FLAG_OWN_CAR.replace(car)

realty = {'Y':0, 'N':1}
df_selected.loc[:, 'FLAG_OWN_REALTY'] = df_selected.FLAG_OWN_REALTY.replace(realty)

# add ROTEM'S code for the last binary column - NAME_CONTRACT_TYPE

city = {1:'A', 2:'B', 3:'C'}
df_selected.loc[:, 'REGION_RATING_CLIENT_W_CITY'] = df_selected.REGION_RATING_CLIENT_W_CITY.replace(city)

In [8]:
df_selected = pd.get_dummies(df_selected, columns=['NAME_HOUSING_TYPE','REGION_RATING_CLIENT_W_CITY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS'])

In [9]:
df_selected.fillna(0, inplace=True)

In [11]:
df_selected.to_csv('./csv/df_app.csv', index=True)

In [12]:
df_selected

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_LAST_PHONE_CHANGE,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,REGION_RATING_CLIENT_W_CITY_A,REGION_RATING_CLIENT_W_CITY_B,REGION_RATING_CLIENT_W_CITY_C,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Secondary,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_not married
0,100002,1,0,1,0,0,202500.0,406597.5,24700.5,-9461,-637,-1134.0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1
1,100003,0,1,1,1,0,270000.0,1293502.5,35698.5,-16765,-1188,-828.0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0
2,100004,0,0,0,0,0,67500.0,135000.0,6750.0,-19046,-225,-815.0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1
3,100006,0,1,1,0,0,135000.0,312682.5,29686.5,-19005,-3039,-617.0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0
4,100007,0,0,1,0,0,121500.0,513000.0,21865.5,-19932,-3038,-1106.0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,1,1,0,157500.0,254700.0,27558.0,-9327,-236,-273.0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1
307507,456252,0,1,1,0,0,72000.0,269550.0,12001.5,-20775,365243,0.0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
307508,456253,0,1,1,0,0,153000.0,677664.0,29979.0,-14966,-7921,-1909.0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1
307509,456254,1,1,1,0,0,171000.0,370107.0,20205.0,-11961,-4786,-322.0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0


### **bureau.csv**

In [22]:
df_col = df_bu.drop(['CREDIT_CURRENCY', 'CREDIT_DAY_OVERDUE', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM_LIMIT',
                  'AMT_CREDIT_SUM_OVERDUE', ], axis=1)

In [23]:
semi_final_df = df_col.groupby(['SK_ID_CURR']).agg( 
    BU_NUM_LOAN = pd.NamedAgg(column= 'SK_ID_BUREAU', aggfunc= 'count'),
    BU_SUM_LOAN = pd.NamedAgg(column= 'AMT_CREDIT_SUM', aggfunc= 'sum'),
    BU_SUM_OPEN_DEBT = pd.NamedAgg(column= 'AMT_CREDIT_SUM_DEBT', aggfunc= 'sum'))

In [24]:
index_drop=df_bu.loc[df_bu['CREDIT_ACTIVE'].isin(['Sold', 'Bad debt'])]
df_bu=df_bu.drop(index_drop.index,axis=0)

df_credit_active=df_bu.groupby(['SK_ID_CURR','CREDIT_ACTIVE']).size()/df_bu.groupby('SK_ID_CURR').size()
df1=pd.DataFrame(df_credit_active).reset_index()
df1=df1.rename(columns={0:'percentage'})

df_final1 = pd.crosstab(index=df1.SK_ID_CURR, 
                   columns=df1.CREDIT_ACTIVE,
                   values=df1.percentage,
                   aggfunc='first').fillna(0)

df_final = pd.concat( [semi_final_df, df_final1] ,axis = 1, join='inner')

In [14]:
df_final.to_csv('./csv/df_bureau.csv', index=True)

In [25]:
df_final

,BU_NUM_LOAN,BU_SUM_LOAN,BU_SUM_OPEN_DEBT,Active,Closed
SK_ID_CURR,,,,,
100001,7,1453365.000,596686.500,0.428571,0.571429
100002,8,865055.565,245781.000,0.250000,0.750000
100003,4,1017400.500,0.000,0.250000,0.750000
100004,2,189037.800,0.000,0.000000,1.000000
100005,3,657126.000,568408.500,0.666667,0.333333
...,...,...,...,...,...
456249,13,3693858.660,163071.000,0.153846,0.846154
456250,3,3086459.550,2232040.095,0.666667,0.333333
456253,4,3960000.000,1795833.000,0.500000,0.500000


### **previous_application.csv**

In [25]:
df_prev=df_prev_new.drop(columns= [ 'AMT_DOWN_PAYMENT',
                                'AMT_GOODS_PRICE',
                                'WEEKDAY_APPR_PROCESS_START',
                                'HOUR_APPR_PROCESS_START',
                                'FLAG_LAST_APPL_PER_CONTRACT',
                                'NFLAG_LAST_APPL_IN_DAY',
                                'RATE_INTEREST_PRIMARY',
                                'RATE_INTEREST_PRIVILEGED',
                                'NAME_CASH_LOAN_PURPOSE',
                                'NAME_TYPE_SUITE',
                                'NAME_CLIENT_TYPE',
                                'NAME_GOODS_CATEGORY',
                                'NAME_PORTFOLIO',
                                'NAME_PRODUCT_TYPE',
                                'CHANNEL_TYPE',
                                'SELLERPLACE_AREA',
                                'NAME_SELLER_INDUSTRY',
                                'NAME_YIELD_GROUP',
                                'PRODUCT_COMBINATION',
                                'DAYS_FIRST_DRAWING',
                                'DAYS_FIRST_DUE',
                                'DAYS_LAST_DUE_1ST_VERSION',
                                'DAYS_LAST_DUE',
                                'NFLAG_INSURED_ON_APPROVAL',
                                'DAYS_DECISION',
                                'NAME_PAYMENT_TYPE',
                                'CODE_REJECT_REASON',
                                'CNT_PAYMENT',
                                'DAYS_TERMINATION',
                                'NAME_CONTRACT_TYPE'])

In [26]:
mask=df_prev[df_prev['AMT_APPLICATION']==0].index
df_prev=df_prev.drop(mask, axis=0)

In [27]:
#aggregation for name contract status column
contract_status=df_prev.groupby(['SK_ID_CURR','NAME_CONTRACT_STATUS']).size()/df_prev.groupby('SK_ID_CURR').size()
df_contract_status=pd.DataFrame(contract_status).reset_index()
df_contract_status=df_contract_status.rename(columns={0:'percentage'})
df_contract_status_agg = pd.crosstab(index=df_contract_status.SK_ID_CURR, 
                                    columns=df_contract_status.NAME_CONTRACT_STATUS,
                                    values=df_contract_status.percentage,
                                    aggfunc='first').fillna(0)
# Everyone who letf has contract status =  'approved'

In [28]:
df_prev['Percentage_approval_request']= np.divide(df_prev.AMT_CREDIT,df_prev.AMT_APPLICATION)

In [29]:
df_final=df_prev.groupby('SK_ID_CURR').agg(
                          mean_anuuity=('AMT_ANNUITY', 'mean'),
                          mean_percent_approval=('Percentage_approval_request', 'mean'),
                          mean_amt_application=('AMT_APPLICATION','mean'),
                          mean_down_pay=('RATE_DOWN_PAYMENT', 'mean'))

In [30]:
df_final.fillna(0, inplace=True)

In [35]:
df_final.to_csv('./csv/df_prev.csv', index=True)

In [31]:
df_final

,mean_anuuity,mean_percent_approval,mean_amt_application,mean_down_pay
SK_ID_CURR,,,,
100001,3951.0000,0.957782,24835.50,0.104326
100002,9251.7750,1.000000,179055.00,0.000000
100003,56553.9900,1.057664,435436.50,0.050030
100004,5357.2500,0.828021,24282.00,0.212008
100005,4813.2000,0.899950,44617.50,0.108964
...,...,...,...,...
456251,6605.9100,1.000000,40455.00,0.000000
456252,10074.4650,0.986561,57595.50,0.062443
456253,4770.4050,0.831397,24162.75,0.214316


### **POS_CASH_balance.csv**

In [16]:
MONTHS_BALANCE_max=df_pos_new.groupby(['SK_ID_PREV'])['MONTHS_BALANCE'].max()
df=pd.DataFrame()
df['SK_ID_PREV']=MONTHS_BALANCE_max.index
df['MONTHS_BALANCE']=MONTHS_BALANCE_max.values

In [20]:
df_POS_CASH_agg=df.merge(df_pos_new, how='inner', on=['SK_ID_PREV','MONTHS_BALANCE'])
df_POS_CASH_agg=df_POS_CASH_agg.drop(columns=['MONTHS_BALANCE','SK_DPD','CNT_INSTALMENT_FUTURE'])

In [21]:
NAME_CONTRACT_STATUS_drop=['Returned to the store', 'Signed', 'Approved', 'Canceled']

In [22]:
index_drop=df_POS_CASH_agg.loc[df_POS_CASH_agg['NAME_CONTRACT_STATUS'].isin(NAME_CONTRACT_STATUS_drop)]
df_POS_CASH_agg=df_POS_CASH_agg.drop(index_drop.index,axis=0)

In [23]:
Debt_rows=df_POS_CASH_agg[df_POS_CASH_agg['NAME_CONTRACT_STATUS'].isin(['Amortized debt', 'Demand'])]
df_POS_CASH_agg.at[Debt_rows.index,'NAME_CONTRACT_STATUS']='Debt'

In [24]:
df_status=df_POS_CASH_agg.groupby(['SK_ID_CURR','NAME_CONTRACT_STATUS']).size()/df_POS_CASH_agg.groupby('SK_ID_CURR').size()
df1=pd.DataFrame(df_status).reset_index()
df1=df1.rename(columns={0:'percentage'})

In [25]:
df_final = pd.crosstab(index=df1.SK_ID_CURR, 
                   columns=df1.NAME_CONTRACT_STATUS,
                   values=df1.percentage,
                   aggfunc='first').fillna(0)

In [26]:
df_final=df_final.reset_index()

In [27]:
df_final.to_csv('./csv/df_pos.csv', index=True)

### **installments_payments.csv:**

In [36]:
df_inst_new.sort_values(by = 'SK_ID_PREV').head(10)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
512588,1000001,158271,1.0,1,-268.0,-294.0,6404.310,6404.310
2159480,1000001,158271,2.0,2,-238.0,-244.0,62039.115,62039.115
1006868,1000002,101962,2.0,4,-1510.0,-1554.0,18443.565,18443.565
1214732,1000002,101962,1.0,1,-1600.0,-1611.0,6264.000,6264.000
3411021,1000002,101962,1.0,3,-1540.0,-1559.0,6264.000,6264.000
1631862,1000002,101962,1.0,2,-1570.0,-1575.0,6264.000,6264.000
4279518,1000003,252457,1.0,3,-34.0,-49.0,4951.350,4951.350
7054246,1000003,252457,1.0,2,-64.0,-81.0,4951.350,4951.350
6145031,1000003,252457,1.0,1,-94.0,-108.0,4951.350,4951.350
7126740,1000004,260094,1.0,1,-862.0,-881.0,3391.110,3391.110


In [27]:
df_inst_new['SK_ID_CURR'].nunique()

334250

In [28]:
df_inst_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9359845 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   SK_ID_CURR              int64  
 2   NUM_INSTALMENT_VERSION  float64
 3   NUM_INSTALMENT_NUMBER   int64  
 4   DAYS_INSTALMENT         float64
 5   DAYS_ENTRY_PAYMENT      float64
 6   AMT_INSTALMENT          float64
 7   AMT_PAYMENT             float64
dtypes: float64(5), int64(3)
memory usage: 642.7 MB


In [29]:
# How many prev loans each person took:
df_inst_new.groupby(['SK_ID_CURR'])['SK_ID_PREV'].nunique().value_counts() #.sum() - How many prev loans exist in the data

1     107917
2      86244
3      56948
4      34681
5      20861
6      11942
7       6826
8       3777
9       2131
10      1247
11       717
12       406
13       232
14       128
15        76
16        47
17        33
18        16
19        15
21         2
23         1
20         1
25         1
22         1
Name: SK_ID_PREV, dtype: int64

* We can see that most of the people took between 1-6 loans, and at the most 26.

In [30]:
df_inst_new.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        1153
AMT_INSTALMENT               0
AMT_PAYMENT               1153
dtype: int64

* Each person who has missing values in DAYS_ENTRY_PAYMENT column has also missing values in the AMT_PAYMENT column.

In [37]:
# What is the delay of installment payment? 
# Negative values mean the person had a debt. positive values mean he paid over his due.
# What is the differnece in installment amount payment?
# Negative values mean the person paid in delya. positive values mean he paid precendently the time.
df_inst_new['diff_AMT'] = df_inst_new['AMT_PAYMENT'] - df_inst_new['AMT_INSTALMENT']
df_inst_new['diff_DAYS'] = df_inst_new['DAYS_ENTRY_PAYMENT'] - df_inst_new['DAYS_INSTALMENT']

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
df_inst_new['diff_DAYS'].value_counts().sort_values()

 1817.0         1
 1866.0         1
 1395.0         1
 1168.0         1
 1186.0         1
            ...  
-9.0       376210
-3.0       380440
-2.0       422147
-1.0       437531
 0.0       604707
Name: diff_DAYS, Length: 1917, dtype: int64

In [8]:
# What is number of installments each laon has?
a = df_inst_new.groupby('SK_ID_CURR')
b = a['SK_ID_PREV'].value_counts()
b

SK_ID_CURR  SK_ID_PREV
100001      1369693        4
            1851984        3
100002      1038818       19
100003      2396755       12
            1810518        7
                          ..
456255      1179690       18
            1743609       10
            1359084        9
            2073384        5
            2729207        3
Name: SK_ID_PREV, Length: 997752, dtype: int64

We wish to choose what are the minimum columns needed to fully describe the table content:
* How many loans each person took
* How many instalments in each loans in average
* What is the mean amount of installments payment
* What is the mean delay of installments payment
* What is the mean differnece in installments amount payment
* What is the percentage of unpaid loans per person?

In [31]:
df_inst_new.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT', 'diff_AMT', 'diff_DAYS'],
      dtype='object')

In [39]:
grouped = df_inst_new.drop(columns = ['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER',\
    'DAYS_INSTALMENT','AMT_INSTALMENT']).groupby('SK_ID_CURR', as_index=False)

grouped = grouped.agg(
    NUM_LOANS=('SK_ID_PREV', 'nunique'),
    MEAN_NUM_INSTALMENT = ('SK_ID_PREV', lambda x: x.count().sum()/x.nunique()),
    MEAN_PAYMENT=('AMT_PAYMENT', 'mean'),
    MEAN_PAY_DIFF=('diff_AMT', 'mean'),
    MEAN_PAY_DELAY=('diff_DAYS', 'mean'),
    NUM_UNPAID_LOANS=('DAYS_ENTRY_PAYMENT', lambda x: x.isna().mean())
)

In [40]:
grouped = grouped.round({'MEAN_PAYMENT':0,\
    'MEAN_PAY_DELAY': 0,\
    'MEAN_PAY_DIFF': 0,\
    'MEAN_NUM_INSTALMENT':2,\
    'NUM_UNPAID_LOANS': 2})
grouped

,SK_ID_CURR,NUM_LOANS,MEAN_NUM_INSTALMENT,MEAN_PAYMENT,MEAN_PAY_DIFF,MEAN_PAY_DELAY,NUM_UNPAID_LOANS
0,100001,1,4.00,7313.0,0.0,-16.0,0.0
1,100002,1,19.00,11559.0,0.0,-20.0,0.0
2,100003,3,8.33,64755.0,0.0,-7.0,0.0
3,100004,1,3.00,7096.0,0.0,-8.0,0.0
4,100005,1,9.00,6240.0,0.0,-24.0,0.0
...,...,...,...,...,...,...,...
334245,456251,1,7.00,7493.0,0.0,-36.0,0.0
334246,456252,1,6.00,10070.0,0.0,-3.0,0.0
334247,456253,2,5.50,4482.0,-361.0,-18.0,0.0
334248,456254,2,9.50,10240.0,0.0,-19.0,0.0


In [41]:
grouped.isna().sum()

SK_ID_CURR              0
NUM_LOANS               0
MEAN_NUM_INSTALMENT     0
MEAN_PAYMENT           12
MEAN_PAY_DIFF          12
MEAN_PAY_DELAY         12
NUM_UNPAID_LOANS        0
dtype: int64

In [43]:
mask=grouped[grouped['MEAN_PAYMENT'].isna()].index
grouped=grouped.drop(mask, axis=0)

In [45]:
# Create new CSV file:
grouped.to_csv('./csv/df_inst.csv', index=True)